In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import tensorflow as tf

os.chdir('D:/SEM 6/ML/PPMI')

In [2]:
file_path = r'D:/SEM 6/ML/PPMI/Health_Control_nifti/PPMI/3000/AX_T2_FLAIR/2011-02-01_08_05_22.0/S102118/PPMI_3000_MR_AX_T2_FLAIR__br_raw_20110322143440789_17_S102118_I224561.nii'

In [3]:
img = nib.load(file_path)
img_data = np.asanyarray(img.dataobj)
img_data = np.resize(img_data, (256,256))
print(img_data.shape)

(256, 256)


In [11]:
x = []
y = []

main1 = 'D:/SEM 6/ML/PPMI/Health_Control_nifti/PPMI'
for sample in os.listdir(main1):
  main2 = main1 + '/' + sample
  for f1 in os.listdir(main2):
    main3 = main2 + '/' + f1
    for f2 in os.listdir(main3):
      main4 = main3 + '/' + f2
      for f3 in os.listdir(main4):
        main5 = main4 + '/' + f3
        for f4 in os.listdir(main5):
            path = main5 + '/' + f4
            img = nib.load(path)
            img_data = np.asanyarray(img.dataobj)
            img_data = np.resize(img_data, (256,256,3))
            x.append(img_data)
            y.append(0)
            
main1 = 'D:/SEM 6/ML/PPMI/PD_NiFTI/PPMI'
for sample in os.listdir(main1):
  main2 = main1 + '/' + sample
  for f1 in os.listdir(main2):
    main3 = main2 + '/' + f1
    for f2 in os.listdir(main3):
      main4 = main3 + '/' + f2
      for f3 in os.listdir(main4):
        main5 = main4 + '/' + f3
        for f4 in os.listdir(main5):
            path = main5 + '/' + f4
            img = nib.load(path)
            img_data = np.asanyarray(img.dataobj)
            img_data = np.resize(img_data, (256,256,3))
            x.append(img_data)
            y.append(1)

In [12]:
print(x[0].shape," ",len(y))

(256, 256, 3)   301


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.2)

In [14]:
X_train = np.array(X_train).reshape(240, 256, 256, 3)
X_test = np.array(X_test)
X_test = np.array(X_test).reshape(61, 256, 256, 3)

In [15]:
print(np.array(y_train).shape)

(240,)


In [16]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [17]:
import tensorflow as tf
from keras.models import Sequential

from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.optimizers import Adam, SGD

from keras import backend as K

In [18]:
data_shape = (256,256,3)

kernel_size = (2, 2)

n_classes = 2

filters = 32

In [19]:
K.clear_session()
model = Sequential()

model.add(Conv2D(filters, kernel_size, activation='softmax', input_shape=data_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters, kernel_size, activation='softmax'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters*2, kernel_size, activation='softmax'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(64, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 255, 255, 32)      416       
_________________________________________________________________
batch_normalization_1 (Batch (None, 255, 255, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 32)      4128      
_________________________________________________________________
batch_normalization_2 (Batch (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 62, 62, 64)       

In [21]:
nEpochs = 50  # Increase this value for better results (i.e., more training)
batch_size = 30  # Increasing this value might speed up fitting

In [22]:
model.fit(np.array(X_train),y_train, epochs=nEpochs, batch_size=batch_size)

Epoch 1/50
240/240 [==============================] - 59s 247ms/step - loss: 0.6699 - accuracy: 0.7208
Epoch 2/50
240/240 [==============================] - 58s 242ms/step - loss: 0.6582 - accuracy: 0.7500
Epoch 3/50
240/240 [==============================] - 58s 243ms/step - loss: 0.6409 - accuracy: 0.7479
Epoch 4/50
240/240 [==============================] - 58s 242ms/step - loss: 0.6553 - accuracy: 0.7458
Epoch 5/50
240/240 [==============================] - 59s 244ms/step - loss: 0.6412 - accuracy: 0.7479
Epoch 6/50
240/240 [==============================] - 59s 245ms/step - loss: 0.6439 - accuracy: 0.7479
Epoch 7/50
240/240 [==============================] - 58s 242ms/step - loss: 0.6451 - accuracy: 0.7292
Epoch 8/50
240/240 [==============================] - 59s 245ms/step - loss: 0.6444 - accuracy: 0.7250
Epoch 9/50
240/240 [==============================] - 58s 242ms/step - loss: 0.6420 - accuracy: 0.7208
Epoch 10/50
240/240 [==============================] - 57s 238ms/step - l

In [23]:
score = model.evaluate(np.array(X_test), y_test, batch_size=3)
print(f'Test loss: {score[0]} / Test Accuracy: {score[1]}')

61/61 [==============================] - 4s 73ms/step
Test loss: 0.5339267336931385 / Test Accuracy: 0.7540983557701111
